I would like to start a bakery business in my neighborhood. As people usually goes to the closest bakery, I am going to use Foursquare location data to know where are the bakeries located and which is the valuation the clients give to them.

In [26]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [27]:
CLIENT_ID = 'C4SZSVGBDB1UCZ1ZK30ZSJSBJMLRJR41VOUL44OORKGQ5HRU' # your Foursquare ID
CLIENT_SECRET = 'FOE4LUCLCUZVEE1FI2P5S4MZOVWNAJUJNHX2PTTM343QQFRA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C4SZSVGBDB1UCZ1ZK30ZSJSBJMLRJR41VOUL44OORKGQ5HRU
CLIENT_SECRET:FOE4LUCLCUZVEE1FI2P5S4MZOVWNAJUJNHX2PTTM343QQFRA


In [37]:
latitude = 39.4483973
longitude = -0.3888961
radius = 1000

In [38]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=C4SZSVGBDB1UCZ1ZK30ZSJSBJMLRJR41VOUL44OORKGQ5HRU&client_secret=FOE4LUCLCUZVEE1FI2P5S4MZOVWNAJUJNHX2PTTM343QQFRA&ll=39.4483973,-0.3888961&v=20180604&radius=1000&limit=30'

In [39]:
import requests

In [40]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 29 around Ecco restaurant.'

In [42]:
items = results['response']['groups'][0]['items']
items[2]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4cadc19c8b57a1cd3e4d9a75',
  'name': 'Arrocería Duna',
  'location': {'address': 'Paseo Pintor Francisco Lozano, MÓDULO 3',
   'crossStreet': 'Tallafoc de la Rambla',
   'lat': 39.453067973,
   'lng': -0.391538462,
   'labeledLatLngs': [{'label': 'display',
     'lat': 39.453067973,
     'lng': -0.391538462}],
   'distance': 567,
   'postalCode': '46012',
   'cc': 'ES',
   'city': 'El Saler',
   'state': 'Comunidad Valenciana',
   'country': 'España',
   'formattedAddress': ['Paseo Pintor Francisco Lozano, MÓDULO 3 (Tallafoc de la Rambla)',
    '46012 El Saler, Valencia Comunidad Valenciana',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1ce941735',
    'name': 'Seafood Restaurant',
    'pluralName': 'Seafood Restaurants',
    'shortName': 'Seafood',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food

In [45]:
dataframe = json_normalize(items) # flatten JSON

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Espai Rambleta,Performing Arts Venue,Bulevar Sur,ES,Valencia,España,Pío IX,336,"[Bulevar Sur (Pío IX), 46017 Valencia Comunida...","[{'label': 'display', 'lat': 39.44739163680030...",39.447392,-0.392594,46017,Comunidad Valenciana,4f3f89bbe4b07147908c8643
1,Parque La Rambleta,Garden,NaN,ES,NaN,España,NaN,414,[España],"[{'label': 'display', 'lat': 39.4462560981987,...",39.446256,-0.392847,NaN,NaN,4bec2a66b3352d7fa25157d2
2,Arrocería Duna,Seafood Restaurant,"Paseo Pintor Francisco Lozano, MÓDULO 3",ES,El Saler,España,Tallafoc de la Rambla,567,"[Paseo Pintor Francisco Lozano, MÓDULO 3 (Tall...","[{'label': 'display', 'lat': 39.453067973, 'ln...",39.453068,-0.391538,46012,Comunidad Valenciana,4cadc19c8b57a1cd3e4d9a75
3,Churreria Casa Piloto,Café,"Calle Los Pedrones, 9",ES,Valencia,España,NaN,266,"[Calle Los Pedrones, 9, 46017 Valencia Comunid...","[{'label': 'display', 'lat': 39.45007380430486...",39.450074,-0.391106,46017,Comunidad Valenciana,4e993b3e0aaf0a73a1c27e20
4,Restaurante Barbados,Restaurant,Los pedrones 2,ES,Valencia,España,NaN,286,"[Los pedrones 2, 46017 Valencia Comunidad Vale...","[{'label': 'display', 'lat': 39.45039964238783...",39.450400,-0.390993,46017,Comunidad Valenciana,4cff8eda4f56b60cfa3b9d37
5,Cervecería El Barrilito,Brewery,NaN,ES,Valencia,España,NaN,252,"[46017 Valencia Comunidad Valenciana, España]","[{'label': 'display', 'lat': 39.44614130186309...",39.446141,-0.389180,46017,Comunidad Valenciana,4e174d6d62e12640d86a3a64
6,Taberna Amparin,Tapas Restaurant,"Joaquin Navarro, 26",ES,Valencia,España,NaN,546,"[Joaquin Navarro, 26, 46017 Valencia Comunidad...","[{'label': 'display', 'lat': 39.45305708423915...",39.453057,-0.386904,46017,Comunidad Valenciana,4d88ec6399c2a1cd34f570d7
7,Pastelería Cafetería A de Alicia,Coffee Shop,NaN,ES,NaN,España,NaN,776,[España],"[{'label': 'display', 'lat': 39.45519211005141...",39.455192,-0.386843,NaN,NaN,4f3e8a82e4b055e9699ab566
8,Starbucks,Coffee Shop,Calle San Vicente Mártir,ES,Valencia,España,Calle de la sangre,388,[Calle San Vicente Mártir (Calle de la sangre)...,"[{'label': 'display', 'lat': 39.45163405141611...",39.451634,-0.390588,46017,Comunidad Valenciana,4b730cf0f964a52032982de3
9,Consum,Grocery Store,Doctor Marañón s/n,ES,Valencia,España,NaN,531,"[Doctor Marañón s/n, 46017 Valencia Comunidad ...","[{'label': 'display', 'lat': 39.45250886564131...",39.452509,-0.392030,46017,Comunidad Valenciana,4c6939f48e7c0f477d02df0f


In [46]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map